In [34]:
from math import cos, sin, pi, acos

In [132]:
class Circle:
    def __init__(self, x, y, r, is_outer=False):
        self.is_outer = is_outer
        self.x = x
        self.y = y
        self.r = r
    def area(self):
        return pi * self.r**2
    def intersects(self, other):
        if other.is_outer:
            return (self.x**2 + self.y**2)**0.5 + self.r > other.r
        else:
            return (other.x - self.x)**2 + (other.y - self.y)**2 < (self.r + other.r)**2
    def c(self):
        return self.x, self.y, self.r

In [133]:
outer = Circle(0, 0, 2 / 3**0.5 + 1, True)
s1 = Circle(cos(0 * pi / 3) * 2 / 3**0.5, sin(0 * pi / 3) * 2 / 3**0.5, 1)
s2 = Circle(cos(2 * pi / 3) * 2 / 3**0.5, sin(2 * pi / 3) * 2 / 3**0.5, 1)
s3 = Circle(cos(4 * pi / 3) * 2 / 3**0.5, sin(4 * pi / 3) * 2 / 3**0.5, 1)
circles = [outer, s1, s2, s3]

In [228]:
def get_coords(c1, c2, rn):
    r1, r2 = c1.r, c2.r
    a = (r1**2+r1*rn+r1*r2-r2*rn)/(r1+r2)
    b = ((r1+rn)**2-a**2)**0.5
    # print(a, b)
    v1 = (r1+r2, 0)
    v2 = (c2.x-c1.x, c2.y-c1.y)
    X = r1+r2
    th = acos(v2[0] / X)
    # print(th / pi * 180)
    d = (r1+rn)
    si = acos(a / d) # + cos(a / d)
    r_th = si + th
    # print(d * cos(r_th), d * sin(r_th))
    return c1.x + d * cos(r_th), c1.y + d * sin(r_th) * (-1 if v2[1] < 0 else 1)

In [230]:
get_coords(s1, s2, 1)

(-0.5773502691896255, -1.0000000000000002)

In [233]:
# c1 will always be outer
def comb_area(c1, c2, c3, d=3):
    if not d:
        return 0
    hasouter = c1.is_outer
    # Pick 2 circles, binary search on radius of last circle. Check if new circle intersects with last circle.
    n_r = 0
    n = 2
    new_c = None
    print(c1.c(), c2.c(), c3.c(), d)
    while n > 0.000001:
        while True:
            t_r = n_r + n
            n_c = get_coords(c2, c3, t_r)
            new_c = Circle(*n_c, t_r)
            if new_c.intersects(c1):
                break
            else:
                n_r = t_r
        n /= 2
    print("New circle is:", new_c.c())

    return new_c.area() + comb_area(c1, c2, new_c, d - 1) + comb_area(c1, c3, new_c, d - 1) + comb_area(c2, c3, new_c, d - 1)

In [235]:
t_area = comb_area(outer, s2, s3) #* 3 + comb_area(s1, s2, s3)

(0, 0, 2.1547005383792515) (-0.5773502691896255, 1.0000000000000002, 1) (-0.5773502691896263, -0.9999999999999999, 1) 3
New circle is: (-1.6720288236682928, 8.881784197001252e-16, 0.48267364501953125)
(0, 0, 2.1547005383792515) (-0.5773502691896255, 1.0000000000000002, 1) (-1.6720288236682928, 8.881784197001252e-16, 0.48267364501953125) 2
New circle is: (-1.780512991034026, 0.7116792707179129, 0.2372264862060547)
(0, 0, 2.1547005383792515) (-0.5773502691896255, 1.0000000000000002, 1) (-1.780512991034026, 0.7116792707179129, 0.2372264862060547) 1
New circle is: (-1.7093378900072596, 1.076606368584791, 0.13457679748535156)
(0, 0, 2.1547005383792515) (-1.6720288236682928, 8.881784197001252e-16, 0.48267364501953125) (-1.780512991034026, 0.7116792707179129, 0.2372264862060547) 1
New circle is: (-2.0065415672933575, 0.4696534568003638, 0.09393119812011719)
(-0.5773502691896255, 1.0000000000000002, 1) (-1.6720288236682928, 8.881784197001252e-16, 0.48267364501953125) (-1.780512991034026, 0.711

KeyboardInterrupt: 

In [103]:
(t_area + s1.area() + s2.area() + s3.area()) / outer.area()

0.6461709275204175